# Data Processing
This notebook is used to process the data from the raw datasets to curated final datasets and then loaded into a Knowledge Graph.

## 1. Setup
### 1.1. Importing Libraries
The first step is to import the necessary libraries for data processing and define the paths to the raw datasets and intermediate saved datasets.

In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import re

datadir = "../data/original"
savedir = "../data/processed"

### 1.2. Loading Data
The second step is to load the raw datasets into the notebook. Each of them is accessed and stored in a pandas DataFrame.

In [2]:
# Load population data
xls = pd.ExcelFile(os.path.join(datadir, "pop_estimates_sa2_2016-2017.xlsx"))

# List to hold DataFrames for each sheet
sheet_list = []

# Iterate through each sheet and read it into a DataFrame
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    sheet_list.append(df)

# Concatenate all DataFrames
population_data = pd.concat(sheet_list, ignore_index=True)

In [12]:
# Load all other data
data_distance_all = pd.read_pickle(os.path.join(datadir, "duration_sa2_hospitals.pkl"))
data_distance_shortest = pd.read_pickle(os.path.join(datadir, "duration_sa2_hospital_shortest.pkl"))

hospital_details = pd.read_csv(os.path.join(datadir, "myhospitals-contact-details.csv"), encoding="latin1")

## 2 Preprocessing datasets
### 2.1 Data Preparation

Rename interesting columns in the data.

In [13]:
population_data = population_data.rename({
                        "SA2 code"   :  "SA2_CODE_2016",
                        "SA2 name"   :  "SA2_NAME_2016",
                        "Area (km2)" :  "AREA_SQKM",
                        "2016pr"     :  "NR_OF_PEOPLE_2016",
                        },
                        axis=1
                  )

Create columns containing 5-digit SA2 codes, the percentage of the population living in a certain SA2, and the population density.

In [14]:
tot_nr_people = population_data["NR_OF_PEOPLE_2016"].sum() # 24.19 million on Google

population_data["SA2_5DIG16"]              = [str(i)[0]+str(i)[-4:] for i in population_data["SA2_CODE_2016"]]
population_data["NR_OF_PEOPLE_2016_%"]     = [round((nr/tot_nr_people)*100,2) for nr in population_data["NR_OF_PEOPLE_2016"]]
population_data["POPULATION_DENSITY_2016"] = [population_data.loc[i,"NR_OF_PEOPLE_2016"]/population_data.loc[i,"AREA_SQKM"] for i in range(len(population_data))]

### 2.2 Data cleaning

In [15]:
# .. VERWIJDEREN ALS HIER NIKS IS..

## 3 Merge datasets
### 3.1 Population Distribution and State Area level 2 (SA2) metadata
The SA2 metadata is created by calculating the population distribution for each SA2 area. The population distribution is calculated by dividing the population of each SA2 area by the total population of the state. The age data is binned into 5-year age groups. The metadata is then merged with the SA2 dataset.

In [16]:
# Take a look at the data
population_data.head()

,S/T code,S/T name,GCCSA code,GCCSA name,SA4 code,SA4 name,SA3 code,SA3 name,SA2_CODE_2016,SA2_NAME_2016,...,Net overseas migration,Unnamed: 16,2016pr-2017p%,2016pr-2017pno.,Unnamed: 19,AREA_SQKM,Population density 2017 (persons/km2),SA2_5DIG16,NR_OF_PEOPLE_2016_%,POPULATION_DENSITY_2016
0,1,New South Wales,1RNSW,Rest of NSW,101,Capital Region,10102,Queanbeyan,101021007,Braidwood,...,11,NaN,2.3,89,NaN,3418.4,1.2,11007,0.02,1.154049
1,1,New South Wales,1RNSW,Rest of NSW,101,Capital Region,10102,Queanbeyan,101021008,Karabar,...,44,NaN,-0.6,-55,NaN,7.0,1225.9,11008,0.04,1230.714286
2,1,New South Wales,1RNSW,Rest of NSW,101,Capital Region,10102,Queanbeyan,101021009,Queanbeyan,...,157,NaN,0.9,106,NaN,4.8,2380.9,11009,0.05,2340.625000
3,1,New South Wales,1RNSW,Rest of NSW,101,Capital Region,10102,Queanbeyan,101021010,Queanbeyan - East,...,68,NaN,0.9,45,NaN,13.0,391.7,11010,0.02,388.384615
4,1,New South Wales,1RNSW,Rest of NSW,101,Capital Region,10102,Queanbeyan,101021011,Queanbeyan Region,...,75,NaN,5.0,873,NaN,3054.4,6.0,11011,0.07,5.708159


Find Relevant Columns

In [17]:
print(list(population_data.columns))

['S/T code', 'S/T name', 'GCCSA code', 'GCCSA name', 'SA4 code', 'SA4 name', 'SA3 code', 'SA3 name', 'SA2_CODE_2016', 'SA2_NAME_2016', 'NR_OF_PEOPLE_2016', '2017p', 'Unnamed: 12', 'Natural increase', 'Net internal migration', 'Net overseas migration', 'Unnamed: 16', '2016pr-2017p%', '2016pr-2017pno.', 'Unnamed: 19', 'AREA_SQKM', 'Population density 2017 (persons/km2)', 'SA2_5DIG16', 'NR_OF_PEOPLE_2016_%', 'POPULATION_DENSITY_2016']


In [18]:
columns = [
    'SA2_CODE_2016',
    "SA2_NAME_2016",
    'SA2_5DIG16',
    'AREA_SQKM',
    'NR_OF_PEOPLE_2016',
    'NR_OF_PEOPLE_2016_%',
    'POPULATION_DENSITY_2016',
]

# Filter the table
SA2PopulationData2016 = population_data[columns]
SA2PopulationData2016.head()

,SA2_CODE_2016,SA2_NAME_2016,SA2_5DIG16,AREA_SQKM,NR_OF_PEOPLE_2016,NR_OF_PEOPLE_2016_%,POPULATION_DENSITY_2016
0,101021007,Braidwood,11007,3418.4,3945,0.02,1.154049
1,101021008,Karabar,11008,7.0,8615,0.04,1230.714286
2,101021009,Queanbeyan,11009,4.8,11235,0.05,2340.625000
3,101021010,Queanbeyan - East,11010,13.0,5049,0.02,388.384615
4,101021011,Queanbeyan Region,11011,3054.4,17435,0.07,5.708159


In [21]:
# Save the data as a csv file
file_path = os.path.join(savedir, "SA2PopulationData2016.csv")
SA2PopulationData2016.to_csv(file_path, index=False)

### 3.2 Distances Hospitals - SA2
The next dataset is the hospital distances to SA2 areas. The dataset is created by adding the closest hospital ID to each SA2 area in the `data_distance_shortest` dataset. 

In [81]:
# Take a look at the data
data_distance_all.head()

,SA2_5DIG16,time_to_1,time_to_2,time_to_3,time_to_4,time_to_5,time_to_6,time_to_7,time_to_8,time_to_9,...,time_to_1002,time_to_1003,time_to_1004,time_to_1005,time_to_1006,time_to_1007,time_to_1008,time_to_1009,time_to_1010,time_to_1011
0,11007,158980.478696,55545.678696,55606.278696,55190.178696,55395.378696,11772.694738,159285.978696,32900.469892,18879.961833,...,18921.478696,176771.378696,16621.333137,32815.309343,30294.871355,23233.794482,7661.486679,29297.294482,155286.478696,12762.078696
1,11008,155655.689234,52220.889234,52281.489234,51865.389234,52070.589234,12091.796855,155961.189234,29592.889234,15555.389234,...,15596.689234,173446.589234,16932.096855,29498.889234,27762.141889,19910.789234,4287.389234,25974.289234,151961.689234,9437.289234
2,11009,155495.195328,52060.395328,52120.995328,51704.895328,51910.095328,11943.307067,155800.695328,29432.395328,15394.895328,...,15436.195328,173286.095328,16783.607067,29338.395328,27918.731659,19750.295328,4126.895328,25813.795328,151801.195328,9276.795328
3,11010,155499.714501,52064.914501,52125.514501,51709.414501,51914.614501,11855.214501,155805.214501,29436.914501,15399.414501,...,15440.714501,173290.614501,16695.514501,29342.914501,28018.014647,19754.814501,4131.414501,25818.314501,151805.714501,9281.314501
4,11011,156375.773471,52940.973471,53001.573471,52585.473471,52790.673471,12260.254294,156681.273471,30312.688050,16275.473471,...,16316.773471,174166.673471,17100.554294,30218.973471,28761.886848,20630.873471,5017.888440,26694.373471,152681.773471,10157.373471


In [82]:
# Take a look at the data
data_distance_shortest.head()

,SA2_5DIG16,shortest_time_sec,shortest_time_min
0,11007,1143.589117,20.0
1,11008,374.559538,7.0
2,11009,202.835640,4.0
3,11010,272.384746,5.0
4,11011,1693.438621,29.0


In [83]:
# Initialize a list to store the results
hospital_IDs = []
hospital_distances = []

# Iterate over the values list
for index, row in tqdm(data_distance_all.iterrows()):
    # Remove SA2_5DIG16 from the row
    filtered_row = row.drop("SA2_5DIG16")
    
    # Remove NaN values
    filtered_row = filtered_row.dropna()
    
    if len(filtered_row) == 0:
        hospital_IDs.append("Not found")
        hospital_distances.append("Not found")
        continue
    
    closests_hospitals = []
    closests_distances = []
    for i in range(0, 5):
        # Find the lowest value in the row
        results = filtered_row.idxmin()
        closests_hospitals.append(results.split("_")[2])
        closests_distances.append(str(filtered_row[results]))
        # Drop the lowest value
        filtered_row = filtered_row.drop(results)
        if len(filtered_row) == 0:
            break
        
    hospital_IDs.append(";".join(closests_hospitals))
    hospital_distances.append(";".join(closests_distances))

data_distance_shortest["closest_hospital_IDs"] = hospital_IDs
data_distance_shortest["closest_hospital_distances"] = hospital_distances

# Drop shortest_time_minutes column
data_distance_shortest = data_distance_shortest.drop(columns=["shortest_time_min"])

# Rename SA2_5DIG16 to SA2_5DIG
data_distance_shortest = data_distance_shortest.rename(columns={"SA2_5DIG16": "SA2_5DIG"})

2310it [00:06, 376.47it/s]


In [84]:
# Replace NaN values with "Not found"
data_distance_shortest = data_distance_shortest.fillna("Not found")

# Save the data as csv
file_path = os.path.join(savedir, "HospitalDistance.csv")
data_distance_shortest.to_csv(file_path, index=False)

### 3.3 Hospitals Metadata
The final dataset is the hospitals metadata. The dataset is created by renaming the columns of the `hospital_details` dataset.

In [85]:
# Inspect the data
hospital_details.head()

,Hospital name,Phone number,Street address,Suburb,Postcode,State,Local Hospital Network (LHN),Primary Health Network area (PHN),Website,Description,Sector,Beds,Latitude,Longitude,Hospital_ID
0,Abbotsford Private Hospital,08 9200 6282,61 Cambridge Street,West Leederville,6007,WA,NaN,NaN,www.abbotsfordhospital.com.au,NaN,Private,<50,-31.940992,115.837344,1
1,Adelaide Clinic,08 8269 8100,33 Park Terrace,Gilberton,5081,SA,NaN,NaN,www.adelaideclinic.com.au/,NaN,Private,50-99,-34.906580,138.613167,2
2,Adelaide Day Surgery Pty Ltd,08 8239 4900,18 North Terrace,Adelaide,5000,SA,NaN,NaN,http://www.curagroup.com.au/adelaide-day-surgery,NaN,Private,<50,-34.922409,138.588817,3
3,Adelaide Eye & Laser Centre,08 8274 7000,215 Greenhill Road,Eastwood,5063,SA,NaN,NaN,www.aelc.com.au,NaN,Private,NaN,-34.940420,138.621500,4
4,Adelaide Surgicentre,08 8211 0000,89 King William Street,Kent Town,5067,SA,NaN,NaN,www.asec.net.au,NaN,Private,<50,-34.917691,138.621022,5


In [86]:
# Create mapping
mapping = {
    "Hospital ID": "hospital_ID",
    "Hospital name": "hospital_name",
    "Phone number": "phone_number",
    "Street address": "address",
    "Suburb": "suburb",
    "Postcode": "postcode",
    "State": "state",
    "Local Hospital Network (LHN)": "local_hospital_network",
    "Primary Health Network (PHN)": "primary_health_network",
    "Website": "website",
    "Description": "description",
    "Sector": "sector",
    "Beds": "beds",
    "Lattitude": "latitude",
    "Longitude": "longitude"
}

# Apply mapping
HospitalMetadata = hospital_details.rename(columns=mapping)
HospitalMetadata.head()

,hospital_name,phone_number,address,suburb,postcode,state,local_hospital_network,Primary Health Network area (PHN),website,description,sector,beds,Latitude,longitude,Hospital_ID
0,Abbotsford Private Hospital,08 9200 6282,61 Cambridge Street,West Leederville,6007,WA,NaN,NaN,www.abbotsfordhospital.com.au,NaN,Private,<50,-31.940992,115.837344,1
1,Adelaide Clinic,08 8269 8100,33 Park Terrace,Gilberton,5081,SA,NaN,NaN,www.adelaideclinic.com.au/,NaN,Private,50-99,-34.906580,138.613167,2
2,Adelaide Day Surgery Pty Ltd,08 8239 4900,18 North Terrace,Adelaide,5000,SA,NaN,NaN,http://www.curagroup.com.au/adelaide-day-surgery,NaN,Private,<50,-34.922409,138.588817,3
3,Adelaide Eye & Laser Centre,08 8274 7000,215 Greenhill Road,Eastwood,5063,SA,NaN,NaN,www.aelc.com.au,NaN,Private,NaN,-34.940420,138.621500,4
4,Adelaide Surgicentre,08 8211 0000,89 King William Street,Kent Town,5067,SA,NaN,NaN,www.asec.net.au,NaN,Private,<50,-34.917691,138.621022,5


In [87]:
# Save the data as csv
file_path = os.path.join(savedir, "HospitalMetadata.csv")
HospitalMetadata.to_csv(file_path, index=False)

## 4 Knowledge Graph
### 4.1 Libraries and Functions

In [88]:
from GraphDB import GraphDB

URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "healthcare"  # Change this to your Neo4j database password
DBNAME = "maingraph"
  
graph_handler = GraphDB(uri=URI, user=USER, pwd=PASSWORD, dbname=DBNAME)

ModuleNotFoundError: No module named 'neo4j'

### 4.2 Creating the Knowledge Graph
#### 4.2.1 Creating the Nodes
State areas (level 2)

In [ ]:
file_path = os.path.join(savedir, "SA2PopulationDistribution.csv")
SA2PopulationDistribution = pd.read_csv(file_path)

# Use the GraphDB class to create nodes for all rows in the dataframe
for index, row in tqdm(SA2PopulationDistribution.iterrows()):
    graph_handler.create_sa2(
        sa2_5dig=row["SA2_5DIG16"], 
        sa2_name=row["SA2_name"], 
        area=row["area"])

Hospitals

In [ ]:
file_path = os.path.join(savedir, "HospitalMetadata.csv")
HospitalMetadata = pd.read_csv(file_path)

for index, row in tqdm(HospitalMetadata.iterrows()):
    graph_handler.create_hospital(
        hospital_id=row["hospital_ID"],
        hospital_name=row["hospital_name"],
        phone_number=row["phone_number"],
        address=row["address"],
        suburb=row["suburb"],
        postcode=row["postcode"],
        state=row["state"],
        lhn=row["local_hospital_network"],
        phn=row["primary_health_network"],
        website=row["website"],
        description=row["description"],
        sector=row["sector"],
        beds=row["beds"],
        latitude=row["latitude"],
        longitude=row["longitude"])

#### 4.2.2 Creating the Edges
Hospital - SA2

In [ ]:
file_path = os.path.join(savedir, "HospitalDistance.csv")
HospitalDistance = pd.read_csv(file_path)

file_path = os.path.join(savedir, "AccessibilityEdges.csv")
AccessibilityEdges = pd.read_csv(file_path)

for index, row in tqdm(HospitalDistance.iterrows()):
    sa2_5dig=row["SA2_5DIG"]
    hospital_ids=row["closest_hospital_IDs"].split(";")
    hospital_distances=row["closest_hospital_distances"].split(";")
    for i, hospital_id in enumerate(hospital_ids):
        row = AccessibilityEdges[
            (AccessibilityEdges["from"] == int(sa2_5dig)) &
            (AccessibilityEdges["to"] == int(hospital_id))
        ]
        if len(row) == 0:
            continue
        accessible = row["accessible"].values[0]
        further_than_2h = row["further_than_2h"].values[0]
        graph_handler.add_relation_sa2_hospital(
            sa2_5dig=sa2_5dig,
            hospital_id=hospital_id,
            distance_time=hospital_distances[i]
        )
    

1it [00:00, 26.00it/s]

False
True


NameError: name 'graph_handler' is not defined

#### 4.2.3 Cleaning up

In [ ]:
graph_handler.close()